In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime as dt
import matplotlib.dates as mdates
import unidecode
from plotly.subplots import make_subplots
from get_data import read_data, get_data, download_csv
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd

## Primeiro, fazemos os plots em Matplotlib:

In [ ]:
df_bairros = gpd.read_file('dados/geojs_bairros_rj.json')
df_bairros['NOME'] = [unidecode.unidecode(x.upper().strip()) for x in list(df_bairros['NOME'])]
#good_names = [bairro['properties']['NOME'] for bairro in geom['features']]
#df_bairros['good_names'] = good_names

In [ ]:
dados_pop_rj = pd.read_csv('dados/Pop_2010_Bairros_2020-2.csv', encoding = 'ISO-8859-1')

In [ ]:
dados_pop_rj['NOME'] = [unidecode.unidecode(x.upper().strip()) for x in list(dados_pop_rj['NOME'])]

In [ ]:
dados_covid_rj = pd.read_csv('dados/Dados_indiv_MRJ_covid19_27-4-2020.csv', encoding = 'ISO-8859-1', delimiter = ';')
dados_covid_rj.drop(columns = ['Data_atualização'], inplace = True)

In [ ]:
dados_covid_rj = read_data(dados_covid_rj, fonte = 'prefeitura')

In [ ]:
nomes = list(dados_covid_rj['Bairro'])
dados_covid_rj.drop(columns = ['Bairro'], inplace = True)

In [ ]:
# mudando os nomes para ficarem com o mesmo padrão:
new_names = []
for nome in nomes:
    if nome == 'FREGUESIA-JPA':
        nome = 'FREGUESIA (JACAREPAGUA)'
        #print('a')
    if nome == 'FREGUESIA-ILHA':
        nome  = 'FREGUESIA (ILHA)'
        #print('b')
    new_names.append(nome)

In [ ]:
dados_covid_rj['Bairro'] = new_names

In [ ]:
lst = dados_pop_rj['NOME'].values
lst = [unidecode.unidecode(b.upper().strip()) for b in lst]

In [ ]:
lst_bairros = list(set(dados_covid_rj['Bairro']))

In [ ]:
lst_bairros.remove('FORA DO MUNICÍPIO')

In [ ]:
dados_certos = dados_covid_rj.drop(index = 3651) # nessa linha, há um erro nos dados

In [ ]:
err_tij = dados_covid_rj[dados_covid_rj['Bairro'] == 'TIJUCA']['Data'].tail(15)

In [ ]:
lst_acumul = []
lst_acumul1 = []
lst_acumul2 = []
lst_acumul3 = []

lst_novos = []
for b in lst_bairros:
    b_novos = get_data(df = dados_certos, local = b, fonte = 'prefeitura', T_fim = '26-04-2020', to_print = False)[1]
    b_por_semanas = [sum(b_novos[7*i:7*(i+1) - 1]) for i in range(4)]
    lst_acumul.append(get_data(df = dados_certos, local = b, fonte = 'prefeitura', T_fim = '26-04-2020', to_print = False)[0][-1])
    lst_acumul1.append(get_data(df = dados_certos, local = b, fonte = 'prefeitura', T_fim = '26-04-2020', to_print = False)[0][-8])
    lst_acumul2.append(get_data(df = dados_certos, local = b, fonte = 'prefeitura', T_fim = '26-04-2020', to_print = False)[0][-15])
    lst_acumul3.append(get_data(df = dados_certos, local = b, fonte = 'prefeitura', T_fim = '26-04-2020', to_print = False)[0][-22])

    lst_novos.append(b_por_semanas)
   

In [ ]:
dic_bairros = {x: unidecode.unidecode(x.upper().strip()) for x in df_bairros['NOME'].values}

In [ ]:
pops = list(dados_pop_rj['poptotal'])

In [ ]:
pops_good_order = [dados_pop_rj['poptotal'][list(dados_pop_rj['NOME']).index(x)] for x in list(df_bairros['NOME'])]
df_bairros['pops'] = pops_good_order

In [ ]:
nova_lst_abs = []
nova_lst1_abs = []
nova_lst2_abs = []
nova_lst3_abs = []
nova_semana_0_abs = []
nova_semana_1_abs = []
nova_semana_2_abs = []
nova_semana_3_abs = []

nova_lst = []
nova_lst1 = []
nova_lst2 = []
nova_lst3 = []
nova_semana_0 = []
nova_semana_1 = []
nova_semana_2 = []
nova_semana_3 = []
nnn = []
for i, x in enumerate(df_bairros['NOME'].values):
    if dic_bairros[x] in lst_bairros:
        nnn.append(x)
        n = lst_bairros.index(x)
        m = list(dados_pop_rj['NOME']).index(x)
        nova_lst_abs.append(lst_acumul[n])
        nova_lst1_abs.append(lst_acumul1[n])
        nova_lst2_abs.append(lst_acumul2[n])
        nova_lst3_abs.append(lst_acumul3[n])
        
        nova_semana_0_abs.append(lst_novos[n][0])
        nova_semana_1_abs.append(lst_novos[n][1])
        nova_semana_2_abs.append(lst_novos[n][2])
        nova_semana_3_abs.append(lst_novos[n][3])
        
        nova_lst.append(lst_acumul[n]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_lst1.append(lst_acumul1[n]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_lst2.append(lst_acumul2[n]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_lst3.append(lst_acumul3[n]*1e4/list(dados_pop_rj['poptotal'])[m])
        
        nova_semana_0.append(lst_novos[n][0]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_semana_1.append(lst_novos[n][1]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_semana_2.append(lst_novos[n][2]*1e4/list(dados_pop_rj['poptotal'])[m])
        nova_semana_3.append(lst_novos[n][3]*1e4/list(dados_pop_rj['poptotal'])[m])

        #print(lst_bairros[n] == x, dados_pop_rj['NOME'][m] == x, x)
    else:
        nnn.append(x)
        nova_lst.append(0)
        nova_lst1.append(0)
        nova_lst2.append(0)
        nova_lst3.append(0)
        nova_semana_0.append(0)
        nova_semana_1.append(0)
        nova_semana_2.append(0)
        nova_semana_3.append(0)

        #print(lst_bairros[n] == x)

In [ ]:
df_bairros['Casos'] = nova_lst
df_bairros['Casos1'] = nova_lst1
df_bairros['Casos2'] = nova_lst2
df_bairros['Casos3'] = nova_lst3
df_bairros['CasosSemana0'] = nova_semana_0
df_bairros['CasosSemana1'] = nova_semana_1
df_bairros['CasosSemana2'] = nova_semana_2
df_bairros['CasosSemana3'] = nova_semana_3


In [ ]:
#fig, ax = plt.subplots(1,1)
#ax.axis('off')
#df_bairros[df_bairros['CODBAIRRO'].str.contains('144|151|149')].plot(column = 'Casos3', ax = ax,legend = True, cmap = 'YlOrRd',
#                                                                    vmin = 0, vmax = 30.965391621129324)
#df_bairros[df_bairros['CODBAIRRO'].str.contains('144|151|149')].boundary.plot(ax = ax, color = 'k')
#

In [ ]:
semanas = [['29/03', '05/04'], ['06/04', '12/04'], ['13/04', '19/04'], ['20/04', '26/04']]

In [ ]:
fig1, ax = plt.subplots(1,1, figsize = (20,12))
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size = '5%', pad = None, add_to_figure =False)
vmin = 0
vmax = 7
i = 3
df_bairros.plot(column = f'CasosSemana%s'%(i), ax = ax,legend = True, cmap = 'YlOrRd', vmin = vmin, vmax = vmax) 
df_bairros.boundary.plot(ax = ax, color = 'k')
ax.axis('off')
#plt.clim((0,40))
#plt.colorbar(boundaries = (0,40))
ax.set_title(f'Casos Novos por 10 mil habitantes \n bairros do Rio de Janeiro na semana do dia %s até o dia %s' %(semanas[i][0], semanas[i][1]), fontsize = 25)
#plt.colorbar(plt.gca().get_children()[0], ax=axes.ravel().tolist())
plt.show()


In [ ]:
titles = ['05/04', '12/04', '19/04', '26/04']
titles_semanas = [semana[0] + ' até ' + semana[1] for semana in semanas]

In [ ]:
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2, figsize = (20,15))
eixos = [ax1, ax2, ax3, ax4]
divider4 = make_axes_locatable(ax4)
dividers = [make_axes_locatable(eixo) for eixo in eixos]
caxs = [divider.append_axes('right', size = '5%', pad = 0.1, add_to_figure = (i == 1)) for i, divider in enumerate(dividers)]
fig.suptitle('Casos acumulados por 10 mil habitantes nos bairros do Rio de Janeiro do dia 29/03 até: ', fontsize = 25)
for i, col in enumerate(['Casos3', 'Casos2', 'Casos1', 'Casos']):
    ax = eixos[i]
    ax.set_title(titles[i], fontsize = 22)
    df_bairros.plot(column = col, ax = ax, legend = True, cax = caxs[i], cmap = 'YlOrRd', vmin = vmin, vmax = 31)
    df_bairros.boundary.plot(ax = ax, color = 'grey')
    ax.axis('off')
fig.tight_layout()
#plt.colorbar(plt.gca().get_children()[0], ax=axes.ravel().tolist())
plt.show()


In [ ]:
##from PIL import Image, ImageDraw
##import imageio
##filenames = 6*['imagens/bairros_novos_semana0.png'] + 6*['imagens/bairros_novos_semana1.png'] + 6*['imagens/bairros_novos_semana2.png'] + 6*['imagens/bairros_novos_semana3.png']
##images = []
##for filename in filenames:
##    images.append(imageio.imread(filename))
##imageio.mimsave(r"imagens/bairros_novos_geoplot.gif" , images)

## Fazendo o html para bairros e áreas de planejamento do RJ:

In [ ]:
import plotly.graph_objects as go

In [ ]:
with open("dados/bairros_novo.geojson", encoding = 'ISO-8859-1') as geofile:
    j_file = json.load(geofile)
with open("dados/geojs_AP.json", encoding = 'ISO-8859-1') as geofile_AP:
    j_file_AP = json.load(geofile_AP)
    

In [ ]:
df_bairros['Casos0'] = df_bairros['Casos']

In [ ]:
## áreas de planejamento do rj
AP1 = df_bairros[df_bairros['AREA_PLANE'] == '1']
AP2 = df_bairros[df_bairros['AREA_PLANE'] == '2']
AP3 = df_bairros[df_bairros['AREA_PLANE'] == '3']
AP4 = df_bairros[df_bairros['AREA_PLANE'] == '4']
AP5 = df_bairros[df_bairros['AREA_PLANE'] == '5']
aps = [AP1, AP2, AP3, AP4, AP5]

In [ ]:
df_bairros['casos0_abs'] = df_bairros['Casos0']*df_bairros['pops']/1e4
df_bairros['casos1_abs'] = df_bairros['Casos1']*df_bairros['pops']/1e4
df_bairros['casos2_abs'] = df_bairros['Casos2']*df_bairros['pops']/1e4
df_bairros['casos3_abs'] = df_bairros['Casos3']*df_bairros['pops']/1e4

df_bairros['casossemana0_abs'] = df_bairros['CasosSemana0']*df_bairros['pops']/1e4
df_bairros['casossemana1_abs'] = df_bairros['CasosSemana1']*df_bairros['pops']/1e4
df_bairros['casossemana2_abs'] = df_bairros['CasosSemana2']*df_bairros['pops']/1e4
df_bairros['casossemana3_abs'] = df_bairros['CasosSemana3']*df_bairros['pops']/1e4


In [ ]:
vals_acum = []
vals_novos = []
for i in range(1, 6):
    vals_acum += [[df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casos0_abs'].sum(),
                   df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casos1_abs'].sum(), 
                   df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casos2_abs'].sum(), 
                   df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casos3_abs'].sum()]]
    vals_novos += [[df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casossemana0_abs'].sum(), 
                    df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casossemana1_abs'].sum(), 
                    df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casossemana2_abs'].sum(), 
                    df_bairros[df_bairros['AREA_PLANE'] == str(i)]['casossemana3_abs'].sum()]]    

In [ ]:
pop1  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 1]['poptotal'])
pop2  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 2]['poptotal'])
pop3  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 3]['poptotal'])
pop4  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 4]['poptotal'])
pop5  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 5]['poptotal'])

In [ ]:
pop1, pop2, pop3, pop4, pop5
pops = [pop1, pop2, pop3, pop4, pop5]

In [ ]:
Casos0_AP, Casos1_AP, Casos2_AP, Casos3_AP = [], [], [], []
CasosSemana0_AP, CasosSemana1_AP, CasosSemana2_AP, CasosSemana3_AP = [], [], [], []

In [ ]:
Casos0_AP, Casos1_AP, Casos2_AP, Casos3_AP = [], [], [], []
Casos0_AP_non_rel, Casos1_AP_non_rel, Casos2_AP_non_rel, Casos3_AP_non_rel  = [], [], [], []
CasosSemana0_AP, CasosSemana1_AP, CasosSemana2_AP, CasosSemana3_AP = [], [], [], []

for i in range(163):
    if df_bairros.iloc[i,4] == '1':
        Casos0_AP.append(vals_acum[0][0]*1e4/pop1)
        Casos1_AP.append(vals_acum[0][1]*1e4/pop1)        
        Casos2_AP.append(vals_acum[0][2]*1e4/pop1)        
        Casos3_AP.append(vals_acum[0][3]*1e4/pop1)
        
        CasosSemana0_AP.append(vals_novos[0][0]*1e4/pop1)
        CasosSemana1_AP.append(vals_novos[0][1]*1e4/pop1)
        CasosSemana2_AP.append(vals_novos[0][2]*1e4/pop1)
        CasosSemana3_AP.append(vals_novos[0][3]*1e4/pop1)
    elif df_bairros.iloc[i, 4] == '2':
        Casos0_AP.append(vals_acum[1][0]*1e4/pop2)
        Casos1_AP.append(vals_acum[1][1]*1e4/pop2)        
        Casos2_AP.append(vals_acum[1][2]*1e4/pop2)        
        Casos3_AP.append(vals_acum[1][3]*1e4/pop2)
        
        CasosSemana0_AP.append(vals_novos[1][0]*1e4/pop2)
        CasosSemana1_AP.append(vals_novos[1][1]*1e4/pop2)
        CasosSemana2_AP.append(vals_novos[1][2]*1e4/pop2)
        CasosSemana3_AP.append(vals_novos[1][3]*1e4/pop2)
    elif df_bairros.iloc[i, 4] == '3':
        Casos0_AP.append(vals_acum[2][0]*1e4/pop3)
        Casos1_AP.append(vals_acum[2][1]*1e4/pop3)        
        Casos2_AP.append(vals_acum[2][2]*1e4/pop3)        
        Casos3_AP.append(vals_acum[2][3]*1e4/pop3)
        
        CasosSemana0_AP.append(vals_novos[2][0]*1e4/pop3)
        CasosSemana1_AP.append(vals_novos[2][1]*1e4/pop3)
        CasosSemana2_AP.append(vals_novos[2][2]*1e4/pop3)
        CasosSemana3_AP.append(vals_novos[2][3]*1e4/pop3)
    elif df_bairros.iloc[i, 4] == '4':
        Casos0_AP.append(vals_acum[3][0]*1e4/pop4)
        Casos1_AP.append(vals_acum[3][1]*1e4/pop4)        
        Casos2_AP.append(vals_acum[3][2]*1e4/pop4)        
        Casos3_AP.append(vals_acum[3][3]*1e4/pop4)
        
        CasosSemana0_AP.append(vals_novos[3][0]*1e4/pop4)
        CasosSemana1_AP.append(vals_novos[3][1]*1e4/pop4)
        CasosSemana2_AP.append(vals_novos[3][2]*1e4/pop4)
        CasosSemana3_AP.append(vals_novos[3][3]*1e4/pop4)
    elif df_bairros.iloc[i, 4] == '5':
        Casos0_AP.append(vals_acum[4][0]*1e4/pop5)
        Casos1_AP.append(vals_acum[4][1]*1e4/pop5)        
        Casos2_AP.append(vals_acum[4][2]*1e4/pop5)        
        Casos3_AP.append(vals_acum[4][3]*1e4/pop5)
        
        CasosSemana0_AP.append(vals_novos[4][0]*1e4/pop5)
        CasosSemana1_AP.append(vals_novos[4][1]*1e4/pop5)
        CasosSemana2_AP.append(vals_novos[4][2]*1e4/pop5)
        CasosSemana3_AP.append(vals_novos[4][3]*1e4/pop5)


In [ ]:
df_bairros['Casos0_AP'] = Casos0_AP
df_bairros['Casos1_AP'] = Casos1_AP
df_bairros['Casos2_AP'] = Casos2_AP
df_bairros['Casos3_AP'] = Casos3_AP

df_bairros['CasosSemana0_AP'] = CasosSemana0_AP
df_bairros['CasosSemana1_AP'] = CasosSemana1_AP
df_bairros['CasosSemana2_AP'] = CasosSemana2_AP
df_bairros['CasosSemana3_AP'] = CasosSemana3_AP

In [ ]:
casos0_ap, casossemana0_ap = [], []
casos1_ap, casossemana1_ap = [], []
casos2_ap, casossemana2_ap = [], []
casos3_ap, casossemana3_ap = [], []

for x in range(1,6):
    casos0_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['Casos0_AP'].iloc[0]]
    casos1_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['Casos1_AP'].iloc[0]]
    casos2_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['Casos2_AP'].iloc[0]]
    casos3_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['Casos3_AP'].iloc[0]]
    
    casossemana0_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['CasosSemana0_AP'].iloc[0]]
    casossemana1_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['CasosSemana1_AP'].iloc[0]]
    casossemana2_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['CasosSemana2_AP'].iloc[0]]
    casossemana3_ap += [df_bairros[df_bairros['AREA_PLANE'] == str(x)]['CasosSemana3_AP'].iloc[0]]

In [ ]:
v_acum = []
v_novos = []
for i in range(len(vals_acum[0])):
    v = []
    novos = []
    for j in range(len(vals_acum)):
        v.append(vals_acum[j][i])
        novos.append(vals_novos[j][i])
    v_acum.append(v)
    v_novos.append(novos)

In [ ]:
df_AP = pd.DataFrame({'AP': [1,2,3,4,5], 'Casos0':casos0_ap, 
                      'Casos1':casos1_ap, 'Casos2':casos2_ap, 
                      'Casos3':casos3_ap, 
                      'CasosSemana0':casossemana0_ap, 
                      'CasosSemana1':casossemana1_ap, 
                      'CasosSemana2':casossemana2_ap, 
                      'CasosSemana3':casossemana3_ap,
                      'pops':pops,
                      'casos0_abs': v_acum[0],
                      'casos1_abs': v_acum[1],
                      'casos2_abs': v_acum[2],
                      'casos3_abs': v_acum[3],
                      'casossemana0_abs': v_novos[0],
                      'casossemana1_abs': v_novos[1],
                      'casossemana2_abs': v_novos[2],
                      'casossemana3_abs': v_novos[3]})

In [ ]:
data_novos = [go.Choropleth(z = df_bairros[f'CasosSemana' + str(i)], 
                          geojson = j_file,
                          locations= df_bairros['CODBNUM'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = (i==0),
                          name = '',
                          zmin = 0,
                          zmax = 7,
                          customdata = df_bairros['pops']*df_bairros[f'CasosSemana'+str(i)]/1e4,
 hovertemplate = "%{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df_bairros['CODBNUM'])
                          for i in range(4)]
data_novos_AP = [go.Choropleth(z = df_AP['CasosSemana'+str(i)], 
                          geojson = j_file_AP,
                          locations= df_AP['AP'],
                          featureidkey = 'properties.CODAPNUM',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          name = '',
                          zmin = 0,
                          zmax = 3,
                          visible = False,
            customdata = df_AP[f'casossemana%s_abs'%(i)],
 hovertemplate = "AP: %{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df_AP['AP'])
                          for i in range(4)]

data_acumul = [go.Choropleth(z = df_bairros[f'Casos%s'%(abs(3-i))], 
                          geojson = j_file,
                          locations= df_bairros['CODBNUM'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = False,
                          name = '',
                          zmin = 0,
                          zmax = 31,
                customdata = df_bairros['pops']*df_bairros[f'Casos%s'%(abs(3-i))]/1e4,
 hovertemplate = "%{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}", 
                          text = df_bairros['CODBNUM']
                          )
                          for i in range(4)]

data_acumul_AP = [go.Choropleth(z = df_AP[f'Casos%s'%(abs(3-i))], 
                          geojson = j_file_AP,
                          locations= df_AP['AP'],
                          featureidkey = 'properties.CODAPNUM',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = False,
                          zmin = 0,
                          zmax = 13,
                          name = '',
                customdata = df_AP[f'casos%s_abs'%(abs(3-i))],
 hovertemplate = "AP: %{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df_AP['AP'])
                          for i in range(4)]

    
data = data_novos + data_acumul + data_novos_AP + data_acumul_AP

In [ ]:
dias = ['29/03', '06/04', '13/04', '20/04']
dias_fim = ['05/04', '12/04', '19/04', '26/04']

In [ ]:
titles = [f'Casos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(4)] + [f'Casos <b>acumulados</b> até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(4)] +[f'Casos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(4)] + [f'Casos <br>acumulados</br> até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(4)]
labels = [f'{dias[i]} até {dias_fim[i]}' for i in range(4)] + [f'até {dias_fim[i]}' for i in range(4)] + [f'{dias[i]} até {dias_fim[i]} (AP)' for i in range(4)] + [f'até {dias_fim[i]} (AP)' for i in range(4)]

In [ ]:
updatemenus = list([dict(active = 0,
                         buttons = list([dict(label = labels[i],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data))] },
                                                      {'title': titles[i]}
                                                      ]) for i in range(len(data))])) ])

In [ ]:
%%time
fig = go.Figure(data= data)
fig.update_layout(updatemenus = updatemenus, 
    title = dict(text = 'Casos <b>novos</b> de 29/03 até 05/04 <br> nos bairros do Rio de Janeiro </br>',
                 x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
#with open("Bairro_plots.html", "w") as f:
#    f.write(fig.to_html())